In [2]:
import findspark
findspark.init()
import pyspark
import matplotlib as plt
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import SQLTransformer
spark = SparkSession.builder.appName('FirstProject').getOrCreate()
Graduate =spark.read.csv('Admissions1.csv',header=True)

In [3]:
Graduate.show()

+---+---+-----+-----------------+---+---+----+--------+-----+
|_c0|GRE|TOEFL|University Rating|SOP|LOR|CGPA|Research|Admit|
+---+---+-----+-----------------+---+---+----+--------+-----+
|  0|337|  118|                4|4.5|4.5|9.65|       1|    1|
|  1|324|  107|                4|4.0|4.5|8.87|       1|    1|
|  2|316|  104|                3|3.0|3.5| 8.0|       1|    1|
|  3|322|  110|                3|3.5|2.5|8.67|       1|    1|
|  4|314|  103|                2|2.0|3.0|8.21|       0|    0|
|  5|330|  115|                5|4.5|3.0|9.34|       1|    1|
|  6|321|  109|                3|3.0|4.0| 8.2|       1|    1|
|  7|308|  101|                2|3.0|4.0| 7.9|       0|    0|
|  8|302|  102|                1|2.0|1.5| 8.0|       0|    0|
|  9|323|  108|                3|3.5|3.0| 8.6|       0|    0|
| 10|325|  106|                3|3.5|4.0| 8.4|       1|    0|
| 11|327|  111|                4|4.0|4.5| 9.0|       1|    1|
| 12|328|  112|                4|4.0|4.5| 9.1|       1|    1|
| 13|307

In [4]:
Graduate.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- GRE: string (nullable = true)
 |-- TOEFL: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Admit: string (nullable = true)



In [5]:
cols = ['GRE','TOEFL','University Rating', 'SOP','LOR','CGPA','Research','Admit']
for col_name in cols:
    Graduate = Graduate.withColumn(col_name, Graduate[col_name].cast('float'))
Graduate = Graduate.withColumnRenamed("Admit", "label")

In [6]:
assembler = VectorAssembler().setInputCols([
    'GRE',
    'TOEFL',
    'University Rating',
    'SOP',
    'LOR',
    'CGPA',
    'Research']).setOutputCol('features')

In [7]:
data02 = assembler.transform(Graduate).select("features","label")

In [8]:
data02.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[337.0,118.0,4.0,...|  1.0|
|[324.0,107.0,4.0,...|  1.0|
|[316.0,104.0,3.0,...|  1.0|
|[322.0,110.0,3.0,...|  1.0|
|[314.0,103.0,2.0,...|  0.0|
|[330.0,115.0,5.0,...|  1.0|
|[321.0,109.0,3.0,...|  1.0|
|[308.0,101.0,2.0,...|  0.0|
|[302.0,102.0,1.0,...|  0.0|
|[323.0,108.0,3.0,...|  0.0|
|[325.0,106.0,3.0,...|  0.0|
|[327.0,111.0,4.0,...|  1.0|
|[328.0,112.0,4.0,...|  1.0|
|[307.0,109.0,3.0,...|  0.0|
|[311.0,104.0,3.0,...|  0.0|
|[314.0,105.0,3.0,...|  0.0|
|[317.0,107.0,3.0,...|  0.0|
|[319.0,106.0,3.0,...|  0.0|
|[318.0,110.0,3.0,...|  0.0|
|[303.0,102.0,3.0,...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [9]:
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(data02)
scaledData = scalerModel.transform(data02)

In [10]:
scaledData.show()

+--------------------+-----+--------------------+
|            features|label|      scaledFeatures|
+--------------------+-----+--------------------+
|[337.0,118.0,4.0,...|  1.0|[0.94,0.928571428...|
|[324.0,107.0,4.0,...|  1.0|[0.68,0.535714285...|
|[316.0,104.0,3.0,...|  1.0|[0.52,0.428571428...|
|[322.0,110.0,3.0,...|  1.0|[0.64,0.642857142...|
|[314.0,103.0,2.0,...|  0.0|[0.48,0.392857142...|
|[330.0,115.0,5.0,...|  1.0|[0.8,0.8214285714...|
|[321.0,109.0,3.0,...|  1.0|[0.62,0.607142857...|
|[308.0,101.0,2.0,...|  0.0|[0.36,0.321428571...|
|[302.0,102.0,1.0,...|  0.0|[0.24,0.357142857...|
|[323.0,108.0,3.0,...|  0.0|[0.66,0.571428571...|
|[325.0,106.0,3.0,...|  0.0|[0.7,0.5,0.5,0.62...|
|[327.0,111.0,4.0,...|  1.0|[0.74,0.678571428...|
|[328.0,112.0,4.0,...|  1.0|[0.76,0.714285714...|
|[307.0,109.0,3.0,...|  0.0|[0.34,0.607142857...|
|[311.0,104.0,3.0,...|  0.0|[0.42,0.428571428...|
|[314.0,105.0,3.0,...|  0.0|[0.48,0.464285714...|
|[317.0,107.0,3.0,...|  0.0|[0.54,0.535714285...|


In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [12]:
logit = LogisticRegression(labelCol="label", featuresCol="scaledFeatures", maxIter=10)

# Train model with Training Data
(trainingData, testData) = scaledData.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())
logit = LogisticRegression(labelCol="label", featuresCol="scaledFeatures", maxIter=10)
lrModel = logit.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.show()

279
121
+--------------------+-----+--------------------+--------------------+--------------------+----------+
|            features|label|      scaledFeatures|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
|[294.0,95.0,1.0,1...|  0.0|[0.08,0.107142857...|[2.57985237134773...|[0.92955360244944...|       0.0|
|[295.0,99.0,2.0,2...|  0.0|[0.1,0.25,0.25,0....|[2.06070115396168...|[0.88702445308913...|       0.0|
|[295.0,101.0,2.0,...|  0.0|[0.1,0.3214285714...|[1.70881531491654...|[0.84668256199436...|       0.0|
|[296.0,95.0,2.0,3...|  0.0|[0.12,0.107142857...|[-0.4517908038181...|[0.38893507085566...|       1.0|
|[297.0,96.0,2.0,2...|  0.0|[0.14,0.142857142...|[1.46681619463950...|[0.81257298089951...|       0.0|
|[298.0,92.0,1.0,2...|  0.0|[0.16,0.0,0.0,0.2...|[2.50572097867506...|[0.92454190993412...|       0.0|
|[298.0,98.0,2.0,1...|  0.0|[0.16,0.214285714...|[-0.352728428610

In [14]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
paramGrid = (ParamGridBuilder().addGrid(logit.regParam, [0.01, 0.5, 2.0]).addGrid(logit.elasticNetParam, [0.0, 0.5, 1.0]).addGrid(logit.maxIter, [1, 5, 10]).build())
cv = CrossValidator(estimator=logit, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cvModel1 = cv.fit(trainingData)

In [15]:
print('Model Intercept: ', cvModel1.bestModel.intercept)
weights = cvModel1.bestModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = spark.createDataFrame(weights, ["Feature Weight"])
weightsDF.show()

Model Intercept:  0.3107197405187981
+--------------------+
|      Feature Weight|
+--------------------+
| 0.02279316734241171|
| 0.01929074262757881|
|0.008964533404471916|
|0.004736882858288...|
|0.002372015684115436|
| 0.03336601451312183|
|0.003001095287548...|
+--------------------+



In [16]:
evaluator.evaluate(predictions, {evaluator.metricName : 'areaUnderROC'})

0.8522139160437034

In [17]:
predictions1=cvModel1.transform(testData)

evaluator.evaluate(predictions1)

0.9108683151236342

In [18]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", featuresCol="scaledFeatures")
rfModel = rf.fit(trainingData)
predictions2 = rfModel.transform(testData)
predictions2.printSchema()
evaluator.evaluate(predictions2)

root
 |-- features: vector (nullable = true)
 |-- label: float (nullable = true)
 |-- scaledFeatures: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



0.8913168487636574

In [19]:
evaluator.evaluate(predictions2)

0.8913168487636574

In [20]:
rfModel.featureImportances

SparseVector(7, {0: 0.233, 1: 0.1093, 2: 0.065, 3: 0.0322, 4: 0.0517, 5: 0.4646, 6: 0.0442})

In [21]:
predictions2.show()

+--------------------+-----+--------------------+--------------------+--------------------+----------+
|            features|label|      scaledFeatures|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
|[294.0,95.0,1.0,1...|  0.0|[0.08,0.107142857...|          [20.0,0.0]|           [1.0,0.0]|       0.0|
|[295.0,99.0,2.0,2...|  0.0|[0.1,0.25,0.25,0....|[19.8101946721311...|[0.99050973360655...|       0.0|
|[295.0,101.0,2.0,...|  0.0|[0.1,0.3214285714...|          [20.0,0.0]|           [1.0,0.0]|       0.0|
|[296.0,95.0,2.0,3...|  0.0|[0.12,0.107142857...|          [19.3,0.7]|[0.96500000000000...|       0.0|
|[297.0,96.0,2.0,2...|  0.0|[0.14,0.142857142...|          [20.0,0.0]|           [1.0,0.0]|       0.0|
|[298.0,92.0,1.0,2...|  0.0|[0.16,0.0,0.0,0.2...|          [20.0,0.0]|           [1.0,0.0]|       0.0|
|[298.0,98.0,2.0,1...|  0.0|[0.16,0.214285714...|[19.1101946721311...|[0.

In [22]:
paramGrid = (ParamGridBuilder().addGrid(rf.maxDepth, [2, 4, 6]).addGrid(rf.maxBins, [20, 60]).addGrid(rf.numTrees, [5, 20]).build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)


In [23]:
prediction = cvModel.transform(testData)
evaluator.evaluate(prediction)

0.9015238642898217

In [24]:
cvModel.bestModel.featureImportances

SparseVector(7, {0: 0.2159, 1: 0.0957, 2: 0.0847, 3: 0.0018, 4: 0.0229, 5: 0.5277, 6: 0.0513})

In [25]:
prediction.show()

+--------------------+-----+--------------------+--------------------+--------------------+----------+
|            features|label|      scaledFeatures|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+--------------------+----------+
|[294.0,95.0,1.0,1...|  0.0|[0.08,0.107142857...|[18.5165875190366...|[0.92582937595183...|       0.0|
|[295.0,99.0,2.0,2...|  0.0|[0.1,0.25,0.25,0....|[18.2506300722281...|[0.91253150361140...|       0.0|
|[295.0,101.0,2.0,...|  0.0|[0.1,0.3214285714...|[18.5165875190366...|[0.92582937595183...|       0.0|
|[296.0,95.0,2.0,3...|  0.0|[0.12,0.107142857...|[17.5822748235819...|[0.87911374117909...|       0.0|
|[297.0,96.0,2.0,2...|  0.0|[0.14,0.142857142...|[18.5165875190366...|[0.92582937595183...|       0.0|
|[298.0,92.0,1.0,2...|  0.0|[0.16,0.0,0.0,0.2...|[18.5165875190366...|[0.92582937595183...|       0.0|
|[298.0,98.0,2.0,1...|  0.0|[0.16,0.214285714...|[17.3163173767734...|[0.